In [1]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import string
%matplotlib inline

In [2]:
if torch.cuda.is_available(): 
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3090


# Setting

In [3]:
filename = "All_1"
filename_KFold = '1'
MAX_LEN = 500 #due to the max length of the token [most BERT agree 512 token] => 512 without features or labels in it
# batch size = 16, 32
batch_size = 24

# Load Dataset

In [4]:
df_crime_train = pd.read_csv(f'../Data/K_Fold_Dataset/{filename_KFold}_Train.csv')
df_crime_test = pd.read_csv(f'../Data/K_Fold_Dataset/{filename_KFold}_Test.csv')

# Clean Message Function

In [5]:
def clean_msg(msg):
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    # ลบ space
    msg = re.sub(r' ','',msg)
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    return msg

# Extract New_Data and News_Label

In [6]:
df_crime_train

,News Title,News_Intro,News_Desc,News_All,All_New_Format,Gambling,Murder,Sexual Abuse,Theft/Burglary,Drug,Battery/Assault,Accident,Non-Crime
0,สาวใหญ่ร้องทุกข์ ตร.นครนายก โดนหนุ่มหื่นบุกอนา...,สาวใหญ่ ร้อง ตร.นครนายก ตามจับหนุ่มหื่นบุกรุกแ...,สาวใหญ่ ร้อง ตร.นครนายก ตามจับหนุ่มหื่นบุกรุกแ...,สาวใหญ่ร้องทุกข์ ตร.นครนายก โดนหนุ่มหื่นบุกอนา...,สาวใหญ่ร้องทุกข์ตรนครนายกโดนหนุ่มหื่นบุกอนาจาร...,0,0,1,0,0,0,0,0
1,ร้านยาซีด!ถูกแจ้ง5ข้อหาหนัก ลอบจำหน่าย'ทรามาดอล',ดส.บุกล่อซื้อจับ หนุ่มท่าศาลาลอบจำหน่ายยาทรามา...,เมื่อวันที่ 23 มี.ค. พ.ต.อ.จิรกฤต จารุภัทร์ ผก...,ร้านยาซีด!ถูกแจ้ง5ข้อหาหนัก ลอบจำหน่าย'ทรามาดอ...,ร้านยาซีดถูกแจ้ง5ข้อหาหนักลอบจำหน่ายทรามาดอล ด...,0,0,0,0,1,0,0,0
2,อดีตกำนันกลับจากหาหมอ ขับฟอร์จูนเนอร์ชนจยย.กลา...,อดีตกำนันจากแก่งกระจาน เพชรบุรี ขับฟอร์จูนเนอร...,อดีตกำนันจากแก่งกระจาน เพชรบุรี ขับฟอร์จูนเนอร...,อดีตกำนันกลับจากหาหมอ ขับฟอร์จูนเนอร์ชนจยย.กลา...,อดีตกำนันกลับจากหาหมอขับฟอร์จูนเนอร์ชนจยยกลางแ...,0,0,0,0,0,0,1,0
3,'จะหาได้แต่ละบาทมันเหนื่อย' สาวเซเว่นฯท้อถูกลั...,สาวเซเว่นฯ กุมขมับด้วยความเครียด จยย.ที่หามาอย...,เมื่อวันที่ 24 เม.ย. ร.ต.อ.วรุตต์ ภูมิภักดิ์ ร...,'จะหาได้แต่ละบาทมันเหนื่อย' สาวเซเว่นฯท้อถูกลั...,จะหาได้แต่ละบาทมันเหนื่อยสาวเซเว่นฯท้อถูกลักจย...,0,0,0,1,0,0,0,0
4,สืบภ.6ทลายยาบ้า3.6แสนเม็ด ซุกแผ่นไม้ตบตาเจ้าหน...,ตำรวจสืบสวนภูธรภาค 6 จับกุมเอเย่นต์ยาบ้าเมืองส...,เมื่อวันที่ 28 มิ.ย. พ.ต.อ.สารนัย คงเมือง รองผ...,สืบภ.6ทลายยาบ้า3.6แสนเม็ด ซุกแผ่นไม้ตบตาเจ้าหน...,สืบภ6ทลายยาบ้า36แสนเม็ดซุกแผ่นไม้ตบตาเจ้าหน้าท...,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,ใช้มาตรการพิเศษแทนเอาผิดอาญา 'ม.3'รุมสหบาทา'ม.2',กรมพินิจฯใช้มาตรการพิเศษแทนดำเนินคดีอาญากับนัก...,เมื่อวันที่ 4 ก.พ. ที่กระทรวงยุติธรรม นายสหการ...,ใช้มาตรการพิเศษแทนเอาผิดอาญา 'ม.3'รุมสหบาทา'ม....,ใช้มาตรการพิเศษแทนเอาผิดอาญาม3รุมสหบาทาม2 กรมพ...,0,0,0,0,0,1,0,0
7703,ผอ.ปัดบีบ'ด.ญ.12ขวบ'ลาออก โร่แจ้งความเอาผิดเพจดัง,ผอ.สาวโรงเรียนสังกัดเทศบาลสระบุรี ยืนยันไม่ได้...,จากกรณีโลกออนไลน์วิพากษ์วิจารณ์อย่างดุเดือด หล...,ผอ.ปัดบีบ'ด.ญ.12ขวบ'ลาออก โร่แจ้งความเอาผิดเพจ...,ผอปัดบีบดญ12ขวบลาออกโร่แจ้งความเอาผิดเพจดัง ผอ...,0,0,1,0,0,1,0,0
7704,'ผู้กองจอย'ใจสู้ไม่เสียขวัญ ลั่นขออยู่ต่อช่วยเ...,รอง ผบช.ตชด. เยี่ยมปลอบขวัญให้กำลังใจตำรวจ ตชด...,เมื่อวันที่ 9 ม.ค. ที่ รพ.สิริรัตนรักษ์ ศูนย์ป...,'ผู้กองจอย'ใจสู้ไม่เสียขวัญ ลั่นขออยู่ต่อช่วยเ...,ผู้กองจอยใจสู้ไม่เสียขวัญลั่นขออยู่ต่อช่วยเหลื...,0,0,0,0,0,0,0,0
7705,"""คมนาคม"" แง้มข่าวดี เตรียมคลายล็อก บนขบวนรถไฟฟ...","""ศักดิ์สยาม"" เผยข่าวดี หลังกรมรางฯมีข้อมูลนักร...","""ศักดิ์สยาม"" เผยข่าวดี หลังกรมรางฯมีข้อมูลนักร...","""คมนาคม"" แง้มข่าวดี เตรียมคลายล็อก บนขบวนรถไฟฟ...",คมนาคมแง้มข่าวดีเตรียมคลายล็อกบนขบวนรถไฟฟ้าแก้...,0,0,0,0,0,0,0,1


In [7]:
X_train = df_crime_train.iloc[:,3:4].values
Y_train = df_crime_train.iloc[:,5:].values
X_test = df_crime_test.iloc[:,3:4].values
Y_test = df_crime_test.iloc[:,5:].values

In [8]:
len(Y_train[0])

8

In [9]:
X_train[2979]

array(['เหตุปะทะที่ปัตตานีพบคนร้ายตายเพิ่มอีก 2 รวมเป็น 7 ศพ ยึดปืนได้ 9 กระบอก ความคืบหน้าเหตุปะทะที่ปัตตานี ล่าสุด จนท.เข้าเคลียร์พื้นที่เจอศพคนร้ายอีก 2 ศพ รวมเป็น 7 ศพ ยึดปืนได้ 9 กระบอก และพบฐานปฏิบัติการ ด้าน กอ.รมน. เผยเสียใจแม้ใช้การเจรจา แต่ไร้ผลต้องยึดตามกฎหมายเมื่อเวลา 13.00 น. วันที่ 16 ส.ค.63 พลตรีปราโมทย์ พรหมอินทร์ โฆษก กอ.รมน.ภาค 4 เปิดเผยกรณีเหตุปะทะในพื้นที่ ม.2 ต.กอลำ อ.ยะรัง จ.ปัตตานี ว่า แม่ทัพภาคที่ 4 ได้กำชับเน้นย้ำการปฏิบัติของเจ้าหน้าที่ให้ใช้ความระมัดระวัง และพยายามที่จะบังคับใช้กฎหมาย เพื่อนำคนผิดม... ความคืบหน้าเหตุปะทะที่ปัตตานี ล่าสุด จนท.เข้าเคลียร์พื้นที่เจอศพคนร้ายอีก 2 ศพ รวมเป็น 7 ศพ ยึดปืนได้ 9 กระบอก และพบฐานปฏิบัติการ ด้าน กอ.รมน. เผยเสียใจแม้ใช้การเจรจา แต่ไร้ผลต้องยึดตามกฎหมายเมื่อเวลา 13.00 น. วันที่ 16 ส.ค.63 พลตรีปราโมทย์ พรหมอินทร์ โฆษก กอ.รมน.ภาค 4 เปิดเผยกรณีเหตุปะทะในพื้นที่ ม.2 ต.กอลำ อ.ยะรัง จ.ปัตตานี ว่า แม่ทัพภาคที่ 4 ได้กำชับเน้นย้ำการปฏิบัติของเจ้าหน้าที่ให้ใช้ความระมัดระวัง และพยายามที่จะบังคับใช้กฎหมาย เพื่อนำคนผิดมาลงโทษ แต่การปฏิ

In [10]:
# clean_text_arr = []
# for i in range(len(X_train)):
#     print(i)
#     clean_text = clean_msg(X_train[i][0])
#     temp_arr = []
#     temp_arr.append(clean_text)
#     clean_text_arr.append(temp_arr)
# # print(clean_text_arr)
# X_train = np.array(clean_text_arr)
# print(X_train)
# print(Y_train)
# print(X_train.shape)
# print(Y_train.shape)

# Split data into train,test,val

In [11]:
from skmultilearn.model_selection import iterative_train_test_split
X_train, Y_train, X_val, Y_val = iterative_train_test_split(X_train,Y_train,test_size=0.1111)

In [12]:
X_train_new = np.array([text for sub in X_train for text in sub]) 
X_test_new = np.array([text for sub in X_test for text in sub]) 
X_val_new = np.array([text for sub in X_val for text in sub]) 

In [13]:
X_train_list = X_train_new.tolist()
X_test_list = X_test_new.tolist()
X_val_list = X_val_new.tolist()

In [14]:
# !pip install sentencepiece
from transformers import BertTokenizer
# cstorm125/wangchanberta-base-att-spm-uncased-finetune
# airesearch/wangchanberta-base-att-spm-uncased
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def preprocessing_for_bert(data):

    input_ids = []
    attention_masks = []

    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            #text=text_preprocessing(sent),  
            text=sent,
            add_special_tokens=True,        
            max_length=MAX_LEN,
            truncation=True,             
            padding='max_length',         
            #return_tensors='pt',           
            return_attention_mask=True      
        )
        
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [15]:
all_texts = np.concatenate([X_train_list, X_test_list, X_val_list])

In [16]:
encoded_texts = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_texts]

Token indices sequence length is longer than the specified maximum sequence length for this model (1600 > 512). Running this sequence through the model will result in indexing errors


In [17]:
token_ids = list(preprocessing_for_bert([X_train_new[0]])[0].squeeze().numpy())
print('Original: ', X_train_new[0])
print('\n')
print('Tokenized: ', tokenizer.tokenize(X_train_new[0]))
print('\n')
print('Token IDs: ', token_ids)

print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train_new)
val_inputs, val_masks = preprocessing_for_bert(X_val_new)

Original:  สาวใหญ่ร้องทุกข์ ตร.นครนายก โดนหนุ่มหื่นบุกอนาจารถึงบ้าน ชี้เป็นภัยสังคม สาวใหญ่ ร้อง ตร.นครนายก ตามจับหนุ่มหื่นบุกรุกและอนาจารถึงในรีสอร์ต โชคดีหนีออกมาขอคนช่วยได้ทัน วงจรปิดจับภาพ พบผู้ก่อเหตุมีประวัติถูกจับแล้วประกันตัวออกมา วอน จนท.จัดการเพราะเป็นภัยสังคมเมื่อเวลาประมาณ 15.00 น. ของวันที่ 8 มิถุนายน 2563 ขณะ พ.ต.ต.วีรศักดิ์ ญาณวุฒิโท สว.(สอบสวน) สภ.เมืองนครนายก กำลังปฏิบัติหน้าที่อยู่ได้มีผู้เสียหายเป็นหญิงอายุ 45 ปี เข้าแจ้งความให้ช่วยติดตามผู้ก่อเหตุเป็นชายขี่รถจักรยานยนต์ ยี่ห้อฮอนด้า ซุปเปอร์คลับ สีฟ้า ทะเบียน... สาวใหญ่ ร้อง ตร.นครนายก ตามจับหนุ่มหื่นบุกรุกและอนาจารถึงในรีสอร์ต โชคดีหนีออกมาขอคนช่วยได้ทัน วงจรปิดจับภาพ พบผู้ก่อเหตุมีประวัติถูกจับแล้วประกันตัวออกมา วอน จนท.จัดการเพราะเป็นภัยสังคมเมื่อเวลาประมาณ 15.00 น. ของวันที่ 8 มิถุนายน 2563 ขณะ พ.ต.ต.วีรศักดิ์ ญาณวุฒิโท สว.(สอบสวน) สภ.เมืองนครนายก กำลังปฏิบัติหน้าที่อยู่ได้มีผู้เสียหายเป็นหญิงอายุ 45 ปี เข้าแจ้งความให้ช่วยติดตามผู้ก่อเหตุเป็นชายขี่รถจักรยานยนต์ ยี่ห้อฮอนด้า ซุปเปอร์คลับ สีฟ้า ทะเบียน 1 กค 3422 น

In [18]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_labels = torch.FloatTensor(Y_train)
val_labels = torch.FloatTensor(Y_val)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [19]:
import torch
import torch.nn as nn
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import CamembertConfig


def initialize_model(epochs=10):
    bert_classifier = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=8)
    bert_classifier = nn.DataParallel(bert_classifier)
    bert_classifier.to(device)
    
    
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=2e-5,    
                      betas=(0.9, 0.98), 
                      eps=1e-6,
                      weight_decay=0.1    
                )

    total_steps = len(train_dataloader) * epochs

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps
                )
    
    return bert_classifier, optimizer, scheduler

In [20]:
import random
import time

# Specify loss function
loss_fn = nn.BCELoss()
m = nn.Sigmoid()
training_stats = list()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=10, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            outputs = model(b_input_ids, b_attn_mask)
            logits = outputs.logits

            # Compute loss and accumulate the loss values
            loss = loss_fn(m(logits), b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)

            training_stats.append(
                {
                    'Epoch': epoch_i + 1,
                    'Training_Loss': avg_train_loss,
                    'Valid_Loss': val_loss,
                    'Valid_Accuracy': val_accuracy,
                    'Time_Elapsed': time_elapsed,
                }
            )

            torch.save(model.state_dict(), f'./Model/MBERT_{filename}_epoch{epoch_i+1}.h5')
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            outputs = model(b_input_ids, b_attn_mask)
            logits = outputs.logits

        # Compute loss
        #print(logits)
        #print(b_labels)
        loss = loss_fn(m(logits), b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        # preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        # accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        accuracy = accuracy_thresh(logits.view(-1, 8), b_labels.view(-1, 8))
#         accuracy = accuracy_thresh(logits.view(-1, 9), b_labels.view(-1, 9))
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

def accuracy_thresh(y_pred, y_true, thresh:float=0.4, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: 
        y_pred = y_pred.sigmoid()
    return ((y_pred > thresh) == y_true.byte()).float().mean().item()

In [21]:
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
bert_classifier, optimizer, scheduler = initialize_model(epochs=4)
train(bert_classifier, train_dataloader, val_dataloader, epochs=4, evaluation=True)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.516090   |     -      |     -     |   8.39   
   1    |   40    |   0.389207   |     -      |     -     |   8.04   
   1    |   60    |   0.373991   |     -      |     -     |   8.18   


In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            outputs = model(b_input_ids, b_attn_mask)
            logits = outputs.logits
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = all_logits.sigmoid().cpu().numpy()

    return probs

In [ ]:
# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('Epoch')

# Display the table.
df_stats.sort_values(by=['Valid_Accuracy'], ascending=False)

,Training_Loss,Valid_Loss,Valid_Accuracy,Time_Elapsed
Epoch,,,,
4,0.157732,0.168539,0.937372,122.498965
3,0.180659,0.175106,0.934564,122.080474
2,0.218717,0.199990,0.927509,121.907855
1,0.334912,0.241848,0.920658,121.204345


In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(X_test_new)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

Tokenizing data...


In [ ]:
probs = bert_predict(bert_classifier, test_dataloader)

In [ ]:
from sklearn.metrics import classification_report

# start from 5 since we need to avoid the news_data <Topic,Intro,Desc,All>
df_label_columns = df_crime_train.columns[5:]
label_names = list(df_label_columns)

print(classification_report(Y_test, np.round(probs), target_names=label_names, zero_division=0))
df_label_columns

                 precision    recall  f1-score   support

       Gambling       0.00      0.00      0.00        25
         Murder       0.90      0.84      0.87       256
   Sexual Abuse       0.87      0.78      0.82        68
 Theft/Burglary       0.82      0.65      0.72        77
           Drug       0.83      0.79      0.81       104
Battery/Assault       0.65      0.53      0.58       189
       Accident       0.73      0.68      0.71        72
      Non-Crime       0.84      0.75      0.79       141

      micro avg       0.81      0.70      0.75       932
      macro avg       0.70      0.63      0.66       932
   weighted avg       0.79      0.70      0.74       932
    samples avg       0.68      0.65      0.65       932



Index(['Gambling', 'Murder', 'Sexual Abuse', 'Theft/Burglary', 'Drug',
       'Battery/Assault', 'Accident', 'Non-Crime'],
      dtype='object')